In [1]:
####################################################
#
# T E S T   D A T A   G E N E R A T I O N
#
# this code is not part of mendonca algorithm
# and used only for experiment purpose in
# order to generate cameras and points
# 
####################################################

In [2]:
import numpy as np
import cv2
import scipy.optimize as opt
import math
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

np.set_printoptions(suppress=True)

In [3]:
state = pd.read_csv("state.csv")

In [4]:
rng = np.random.default_rng(3122787423)

In [5]:
def p3t(T,x,y,z):
    # apply a Projective 3D Transform
    xyz = np.concatenate((x, y, z),axis=1)
    column_ones = np.ones((len(x),1))
    tmp = T @ (np.concatenate((xyz,column_ones),axis=1)).T
    xp = (tmp[0,:]/tmp[3,:]).T
    yp = (tmp[1,:]/tmp[3,:]).T
    zp = (tmp[2,:]/tmp[3,:]).T
    return xp,yp,zp

In [6]:
def projf(P,x,y,z):
#     % PROJ  compute perspective projection (from 3D to pixel coordinates)
#     %   pixel positions are returned with floating point precision
#     %
#     %   See also PROJE

    c3d = np.concatenate((x, y, z),axis=1)
    column_ones = np.ones((len(x),1))
    h3d = (np.concatenate((c3d,column_ones),axis=1)).T
    h2d = P @ h3d

    c2d = h2d/ h2d[2,:]

    u = c2d[0,:].T
    v = c2d[1,:].T
    return u,v

In [7]:
def plot_3d(x,y,z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x,y,z)
    plt.show()

In [8]:
def plot_2d(x,y):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x,y)
    plt.show()

In [9]:
def conver_to_col(x):
    return np.reshape(x,(len(x),1))

In [10]:
def rcam(A):
# RCAM generate a random camera
#    generate a random camera pointing to lookp, positioned at an average 
#    distance ad form the origin, with a std dev of sd 
#    A is the intrinsic parameters matrix

    ad=2.5
    sd=0.25
    lookp=np.zeros((1,3))
    eyep = rng.uniform(-1,1,size=(1,3))-0.5
    R = np.zeros((3,3))
    eyep = eyep/np.linalg.norm(eyep) * (ad + sd*rng.random(1))

    R[2,:] = lookp - eyep/np.linalg.norm(lookp - eyep)
    R[1,:] = np.cross(R[2,:],rng.uniform(size=(1,3)))
    R[1,:] = R[1,:]/np.linalg.norm(R[1,:])
    R[0,:] = np.cross(R[1,:],R[2,:])
    Rt = np.concatenate((R,-R @ eyep.T),axis=1)

    return A @ Rt


In [11]:
numberOfViews = 5
numberOfPoints = 50
imagePoints = np.zeros((numberOfPoints,2,numberOfViews))
PPM = np.zeros((3,4,numberOfViews))
PPMGT = np.zeros((3,4,numberOfViews))

In [12]:
def create_fake_points():
    data = (rng.uniform(size=(numberOfPoints,3),)-0.5)/(math.sqrt(3)/2)
    x = data[:,0]
    y = data[:,1]
    z = data[:,2]
    return x, y, z

In [13]:
true_K = np.array([
    [800,0,256],
    [0,800,256],
    [0,0,1]
    ])
x, y, z = create_fake_points()
A = true_K
P = rcam(A)
invAP = np.linalg.inv(A) @ P
lower_line = [[0,0,0,1]]
G0 = np.concatenate((invAP, lower_line),axis=0)

In [14]:
x,y,z = p3t(
    G0,
    conver_to_col(x),
    conver_to_col(y),
    conver_to_col(z)
)
P = A @ np.concatenate((np.identity(3), np.zeros((3,1))),axis=1)

u,v = projf(P,conver_to_col(x),conver_to_col(y),conver_to_col(z))

imagePoints[:,0,0] = u
imagePoints[:,1,0] = v
PPMGT[:,:,0] = P

In [15]:
for view in range (1,numberOfViews):
    # random camera position
    P = rcam(A)
    # apply world coordinate transformation
    P = P @ np.linalg.inv(G0);
    # project world points to image points
    u,v = projf(P,conver_to_col(x),conver_to_col(y),conver_to_col(z))
    imagePoints[:,0,view] = u
    imagePoints[:,1,view] = v
    PPMGT[:,:,view] = P   

# A Simple Technique for Self-Calibration
Alexander Kruglyak

Sofya Zybtsovsky

### Introduction
The goal of Computer Vision is to compute properties (mainly geometric) of the three-dimensional world from images. One of the challenging problems of Computer Vision is to reconstruct a 3D model of a scene from a moving camera. 
Possible applications include: navigation of autonomous vehicles, objects recognition, reverse engineering and synthesis of virtual environments.


In general assumption that the intrinsic parameters of the camera (focal length, image center and aspect ratio) are known. 
Computing camera motion in this case is a well known problem in photogrammetry, called relative orientation, for which several methods are available. 
Given all the parameters of the camera, reconstruction is straightforward.


### Mendonca algorithm
Input: 3+ images such as contain at least 8+ correspondence points

Output:  K (intrinsic  parameters)

1. Init some random matrix Ki for every image 1≤ i ≤ n
2. Calculate fundamental matrix F for each pair of images (for example via 8-point algorithm with data normalization)
3. Calculate essential matrix E for each pair of images (based on F and K)
4. Decompose E and find singular values (SVD)
4. Calculate error (cost function)
6. Minimize error (e.g, via least squares)
7. Then cost function reach global minimum: return K


### 1. Init some random matrix K

In [19]:
initial_K = [true_K[0,0], true_K[0,2],true_K[1,1],true_K[1,2]]

### 2. Calculate fundamental matrix F for each pair of images (for example via 8-point algorithm with data normalization)


In [17]:
def calc_fund_matrix(points1,points2):
    '''
    Takes in 2 arrays of matching points 
    Return Fundamental matrix computes 
    using 8 point algorithm
    '''
#     F, mask = cv2.findFundamentalMat(points1,points2,cv2.FM_8POINT)
    F, mask = cv2.findFundamentalMat(points1,points2)

    return F 

Fs = np.zeros((3,3,numberOfViews,numberOfViews))

for i in range(numberOfViews):
    for j in range(i+1,numberOfViews):
        Fs[:,:,i,j] = calc_fund_matrix(imagePoints[:,:,i],imagePoints[:,:,j])

### 3. Calculate essential matrix E for each pair of images (based on F and K)

In [18]:
def calculate_essential_matrix(K,F):
    return K.T @ F @ K

### 4. Decompose E and find singular values (SVD)

In [19]:
def decompose_singular_values(E):
    _,D,_ = np.linalg.svd(E)
    # Singular Values (3rd value, D[3] is 0 according to theorem)
    r = D[0]
    s = D[1]
    return r, s

### 5. Calculate error (cost function)

1𝞼ij , 2𝞼ij  - non zero singular values of Eij = KTiFijKj, such as 1𝞼ij > 2𝞼ij 

wij - normalized weight factors

Minimize cost function:
<img src="images/cost_function.png">

In [20]:
def mendonca_cost_func(X):
    '''
    computes Mendonca & Cipolla Cost function to find the Optimal Intrinsic Parameters
    Input
    X      - Approximate Values of Intrinsics - 1D array with length 5
    Output
    E    - Computed Cost
    '''

    K = np.array([
        [X[0],0,X[1]],
        [0,X[2],X[3]],
        [0,0,1]
    ])
    cost = 0
    nof_images = numberOfViews
    Den = nof_images*(nof_images-1)/2 

    for i in range(0,nof_images-1):
        for j in range (i+1,nof_images):
            E = calculate_essential_matrix(K, Fs[:,:,i,j])
            r,s = decompose_singular_values(E)
            cost+= (1/Den) * (r - s)/s

    return cost

### 6. Minimize error (e.g, via least squares)

In [21]:
res = opt.minimize(mendonca_cost_func,x0=initial_K, method='Nelder-Mead')

### 7. Then cost function reach global minimum: return K

In [22]:
result_K = np.array([
    [ res.x[0], 0 ,       res.x[1] ],
    [ 0,        res.x[2], res.x[3] ],
    [ 0,        0,        1        ]
])
print (np.matrix(result_K))

[[799.98911309   0.         255.99920977]
 [  0.         799.98828015 255.99955479]
 [  0.           0.           1.        ]]


### Mendonca | Advantages

simple mathematical calculation

find all unknown intrinsic camera parameters

always converges to the global minimum

has no bias towards any particular image of the sequence

insensitive to the initialization


In [417]:
# result_K = np.zeros((3,3))


In [418]:
initial_K = [1,200,1,1]
res = opt.minimize(mendonca_cost_func,x0=initial_K, method='Nelder-Mead')
result_K = np.zeros((3,3))
result_K = np.array([[res.x[0],0,res.x[1]],[0,res.x[2],res.x[3]],[0,0,1]])
print (np.matrix(result_K))

[[  1.   0. 200.]
 [  0.   1.   1.]
 [  0.   0.   1.]]


<ipython-input-414-95c12335e822>:23: RuntimeWarning: invalid value encountered in double_scalars
  cost+= (1/Den) * (r - s)/s
